In [ ]:
"""
ip = "180.166.103.21"
port = 55213
config={
    "OutputElapsed": true, // 将数据进入插件，到插入流表前的时间间隔作为一个字段置于表末尾，类型为 LONG。perPenetrationTime, nanosecond (not the latency we are looking at)
    "ReceivedTime": true, // 将接收到数据的时间作为一个字段置于表末尾，类型为 NANOTIMESTAMP。默认为 true。receivedTime
    "ConcatTime": true // 将 UpdateTime 和 UpdateMillisec 字段合并为一个 tradeTime 字段置于表末尾，类型为 TIME
}
ids = `m2601`y2601
conn = ctp::connect(ip, port, config)



marketTBCapacity=300000
ctpSubType = "marketData"
ctpSubTBName = "ctpMarketDataStream"
tb = ctp::getSchema(conn, ctpSubType) 
 
if (not existsStreamTable(ctpSubTBName)){
    enableTableShareAndPersistence(
        table=streamTable(marketTBCapacity:0, tb.name, tb.typeString), 
        tableName=ctpSubTBName,
        cacheSize=marketTBCapacity,
    preCache=0)
};go

snapshot_coldef_name = `InstrumentID`trade_date`last_price`open_price`high_price`low_price`close_price`volume`turnover`open_interest`pre_settlement_price`pre_close_price`pre_open_interest`pre_delta`upper_limit_price`lower_limit_price`bid_price`bid_volume`ask_price`ask_volume`trade_time`data_time`utc_data_time
snapshot_coldef_type = [SYMBOL,,DATE,DOUBLE,DOUBLE,DOUBLE,DOUBLE,DOUBLE,INT,DOUBLE,DOUBLE,DOUBLE,DOUBLE,DOUBLE,DOUBLE,DOUBLE,DOUBLE,DOUBLE[],INT[],DOUBLE[],INT[],TIME,TIMESTAMP,TIMESTAMP]
minute_coldef_name = `data_time`InstrumentID`trade_date`open`high`low`close`volume`open_interest`turnover`pre_settlement_price`pre_close_price`pre_open_interest`pre_delta`upper_limit_price`lower_limit_price`first_bid_price`first_ask_price`first_bid_volume`first_ask_volume`avg_spread
minute_coldf_type = [TIMESTAMP,SYMBOL,DATE,DOUBLE,DOUBLE,DOUBLE,DOUBLE,INT,DOUBLE,DOUBLE,DOUBLE,DOUBLE,DOUBLE,DOUBLE,DOUBLE,DOUBLE,DOUBLE,DOUBLE,INT,INT,DOUBLE]
day_coldef_name = `data_time`InstrumentID`trade_date`open`high`low`close`volume`open_interest`turnover`pre_settlement_price`pre_close_price`pre_open_interest`pre_delta`upper_limit_price`lower_limit_price`first_bid_price`first_ask_price`first_bid_volume`first_ask_volume`avg_spread
day_coldef_type = [TIMESTAMP,SYMBOL,DATE,DOUBLE,DOUBLE,DOUBLE,DOUBLE,INT,DOUBLE,DOUBLE,DOUBLE,DOUBLE,DOUBLE,DOUBLE,DOUBLE,DOUBLE,DOUBLE,DOUBLE,INT,INT,DOUBLE]


snapshot_table_name = "ctpSnapshot"
min_table_name = "ctpMin"
day_table_name = "ctpDay"
if (not existsStreamTable(snapshot_table_name)){
    enableTableShareAndPersistence(
        table=streamTable(marketTBCapacity:0, snapshot_coldef_name, snapshot_coldef_type), 
        tableName=snapshot_table_name,
        cacheSize=marketTBCapacity,
    preCache=0)
};go
if (not existsStreamTable(min_table_name)){
    enableTableShareAndPersistence(
        table=streamTable(int(marketTBCapacity/10):0, minute_coldef_name, minute_coldf_type), 
        tableName=min_table_name,
        cacheSize=int(marketTBCapacity/10),
    preCache=0)
};go
if (not existsStreamTable(day_table_name)){
    enableTableShareAndPersistence(
        table=streamTable(int(marketTBCapacity/100):0, day_coldef_name, day_coldef_type), 
        tableName=day_table_name,
        cacheSize=int(marketTBCapacity/100),
    preCache=0)
};go

now_day = date(now())
rse_metrics=<[
    iif(tradeTime > 20:00:00.000, temporalAdd(now_day, 1, "d"), now_day) as trade_date,
    LastPrice as last_price,
    OpenPrice as open_price,
    HighestPrice as high_price,
    LowestPrice as low_price,
    ClosePrice as close_price,
    Volume as volume,
    Turnover as turnover,
    OpenInterest as open_interest,
    PreSettlementPrice as pre_settlement_price,
    PreClosePrice as pre_close_price,
    PreOpenInterest as pre_open_interest,
    PreDelta as pre_delta,
    UpperLimitPrice as upper_limit_price,
    LowerLimitPrice as lower_limit_price,
    fixedLengthArrayVector(BidPrice1, BidPrice2, BidPrice3, BidPrice4, BidPrice5) as bid_price,
    fixedLengthArrayVector(BidVolume1, BidVolume2, BidVolume3, BidVolume4, BidVolume5) as bid_volume,
    fixedLengthArrayVector(AskPrice1, AskPrice2, AskPrice3, AskPrice4, AskPrice5) as ask_price,
    fixedLengthArrayVector(AskVolume1, AskVolume2, AskVolume3, AskVolume4, AskVolume5) as ask_volume,
    tradeTime as trade_time,
    temporalParse(string(iif(tradeTime > 20:00:00.000, temporalAdd(now_day, 1, "d"), now_day)) + " " + string(tradeTime), "yyyy.MM.dd HH:mm:ss.SSS") as data_time,
    temporalAdd(temporalParse(string(iif(tradeTime > 20:00:00.000, temporalAdd(now_day, 1, "d"), now_day)) + " " + string(tradeTime), "yyyy.MM.dd HH:mm:ss.SSS"), -8, "H") as utc_data_time
]>

rse = createReactiveStateEngine(name="rse_0", 
        metrics =rse_metrics, 
        dummyTable=objByName(ctpSubTBName), 
        outputTable=objByName(snapshot_table_name), 
        keyColumn="InstrumentID"
)
subscribeTable(,
    ctpSubTBName, 
    "rse_0", 
    handler=tableInsert{rse}, 
    msgAsTable=true
)

tse_min_metrics=<[
    last(trade_date) as trade_date, 
    firstNot(last_price, 0.0) as open,
    max(last_price, high_price) as high,
    min(last_price, low_price) as low, 
    lastNot(last_price, 0.0) as close, 
    last(volume)-first(volume) as volume, 
    last(open_interest) as open_interest, 
    last(turnover) - first(turnover) as turnover, 
    last(pre_settlement_price) as pre_settlement_price, 
    last(pre_close_price) as pre_close_price, 
    last(pre_open_interest) as pre_open_interest, 
    last(pre_delta) as pre_delta, 
    last(upper_limit_price) as upper_limit_price, 
    last(lower_limit_price) as lower_limit_price, 
    first(bid_price[0]) as first_bid_price, 
    first(ask_price[0]) as first_ask_price, 
    first(bid_volume[0]) as first_bid_volume, 
    first(ask_volume[0]) as first_ask_volume, 
    avg(ask_price[0] - bid_price[0]) as avg_spread
]>
tse_day_metrics=<[
    last(trade_date) as trade_date, 
    lastNot(open_price, 0.0) as open,
    lastNot(high_price, 0.0) as high,
    lastNot(low_price, 0.0) as low, 
    lastNot(last_price, 0.0) as close, 
    last(volume) as volume, 
    last(open_interest) as open_interest, 
    last(turnover) as turnover, 
    last(pre_settlement_price) as pre_settlement_price, 
    last(pre_close_price) as pre_close_price, 
    last(pre_open_interest) as pre_open_interest, 
    last(pre_delta) as pre_delta, 
    last(upper_limit_price) as upper_limit_price, 
    last(lower_limit_price) as lower_limit_price, 
    first(bid_price[0]) as first_bid_price, 
    first(ask_price[0]) as first_ask_price, 
    first(bid_volume[0]) as first_bid_volume, 
    first(ask_volume[0]) as first_ask_volume, 
    avg(ask_price[0] - bid_price[0]) as avg_spread
]>
def end_minus_start_time_seconds(startT, endT){
    return iif(endT-startT<0, (endT-startT+12:00:00.000+12:00:00.000) / 1000, (endT-startT)/1000)
}
ss_list = [21:00:00.000, 09:00:00.000, 10:30:00.000, 13:30:00.000]
ee_list = [23:00:00.000, 10:15:00.000, 11:30:00.000, 15:00:00.000]
nSec = 60
num_of_sym = 2
s_start, e_end = ss_list[0], ee_list[int(size(ee_list)-1)]
duration = end_minus_start_time_seconds(s_start, e_end)

tse_min = createDailyTimeSeriesEngine(
    name="tse_min_0", 
    windowSize=1000*nSec, step=1000*nSec, forceTriggerTime=1000*nSec,
    metrics=tse_min_metrics,  
    dummyTable=objByName(snapshot_table_name), 
    outputTable=objByName(min_table_name),
    timeColumn=`data_time, 
    keyColumn=`InstrumentID,
    fill=`ffill,
    sessionBegin=ss_list, 
    sessionEnd=ee_list
)
subscribeTable(,
    tableName=snapshot_table_name, 
    actionName="tse_min_0", 
    handler=tableInsert{tse_min}, 
    msgAsTable=true
)
tse_day = createDailyTimeSeriesEngine(
    name="tse_day_0", 
    windowSize=1000*duration, step=1000*duration, forceTriggerTime=1000*duration,
    metrics=tse_day_metrics,  
    dummyTable=objByName(snapshot_table_name), 
    outputTable=objByName(day_table_name),
    timeColumn=`data_time, 
    keyColumn=`InstrumentID,
    fill=`ffill,
    sessionBegin=s_start, 
    sessionEnd=e_end
)
subscribeTable(,
    tableName=snapshot_table_name, 
    actionName="tse_day_0", 
    handler=tableInsert{tse_day}, 
    msgAsTable=true
)

share streamTable(int(marketTBCapacity/10):0, 
    `data_time`InstrumentID`delta`upper`lower`factor1`factor2, 
    [TIMESTAMP,SYMBOL, DOUBLE, DOUBLE, DOUBLE, INT,INT]
) as result_stream;go

bband_mul = 1
mavg_dur = 3
streamEngine=streamEngineParser(
    name=`alpha0, 
    metrics=<[InstrumentID, 
        byRow(deltas,close),
        mavg(byRow(deltas,close),mavg_dur) - bband_mul* mstd(byRow(deltas,close),mavg_dur),
        mavg(byRow(deltas,close),mavg_dur) + bband_mul* mstd(byRow(deltas,close),mavg_dur),
        iif(isNull(mavg(byRow(deltas,close),mavg_dur)),0,
            iif(byRow(deltas,close) < mavg(byRow(deltas,close),mavg_dur) - bband_mul* mstd(byRow(deltas,close),mavg_dur), -1,
            iif(byRow(deltas,close) > mavg(byRow(deltas,close),mavg_dur) + bband_mul* mstd(byRow(deltas,close),mavg_dur), 1, 0)
        )),
        iif(isNull(mavg(byRow(deltas,close),mavg_dur)),0,
            iif(byRow(deltas,close) < mavg(byRow(deltas,close),mavg_dur) - bband_mul* mstd(byRow(deltas,close),mavg_dur), 1,
            iif(byRow(deltas,close) > mavg(byRow(deltas,close),mavg_dur) + bband_mul* mstd(byRow(deltas,close),mavg_dur), -1, 0)
        ))
    ]>, 
    dummyTable=objByName(min_table_name), outputTable=result_stream, 
    keyColumn=`InstrumentID, timeColumn=`data_time,
    useSystemTime = false,useWindowStartTime = false,
    triggeringPattern='keyCount',triggeringInterval=num_of_sym
)
subscribeTable(tableName=min_table_name, actionName="test_stream", handler=append!{streamEngine}, msgAsTable=true)
go


ctp::subscribe(conn, ctpSubType, objByName(ctpSubTBName), ids);

ctp::close(conn)
clearAllCache()


"""